# Ayudantía 6 - Reacciones Múltiples

Dpto. de Ingeniería Química y Bioprocesos

Diseño de Reactores - IIQ2113

Ayudante: Esteban López Stephan  - esteban.lpez@uc.cl

Para abrir en Google Colab: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/SysBioengLab/IIQ2113-Chemical-reactor-design/blob/main/Ayudantias/Ayudantia6.ipynb)

# Obtención de parametros
### Antes de desarrollar el ejercicio en si, necesitamos todos los parametros cineticos de las reacciones que ocurren, como no nos entregan el valor de $k_1$ tendremos que conseguirlo usando los datos experimentales que nos entregan de esta reacción

## Librerias a importar

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import least_squares
from scipy.integrate import quad
from thermo.unifac import UNIFAC, UFSG, UFIP
from thermo.chemical import Chemical
from ugropy import Groups

Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-19


## Definición de funciones a utilizar
### Para facilitar la definición del sistema de reacción que vamos a evaluar se definiran funciones a continuación

## Determinación de fracciones molares
### Este código determina la fracción molar global de cada componente en el sistema, pudiendo recibir cantidades en masa (g) o en moles


In [ ]:
##función para obtener moles basados en datos experimentales
##La presión debe entregarse en bares y la temperatura en Kelvin
def get_moles(components,components_initial_values,T,P):
    components_moles=[]
    for i in range(len(components_initial_values)):
        if components_initial_values[i][1]=='moles':
            components_moles.append(components_initial_values[i][0])
        elif components_initial_values[i][1]=='mass':
            component=Chemical(components[i],T=T,P=P*100000)
            component_molar_mass=component.MW
            components_moles.append(components_initial_values[i][0]/component_molar_mass)
        else:
            raise ValueError("invalid unit for component initial values. Use 'mass' or 'moles'")
    return components_moles

In [ ]:
#Esta función determina la velocidad de reacción inicial para todos los experimentos entregados
def initial_reaction_rate(experimental_molar_concentrations,kinetic_param):
    #esta función devolverá una lista de velocidades iniciales de reacción para los diferentes experimentos y graficará los resultados
    initial_rx_rates=[]
    number_of_experiments=len(experimental_molar_concentrations)
    for i in range(number_of_experiments):
        K1=kinetic_param
        initial_rate_rx_i=K1*(experimental_molar_concentrations[i][0]**0.5+experimental_molar_concentrations[i][1]**0.5)/experimental_molar_concentrations[i][0]
        #añadimos la velocidad inicial de reacción del experimento i
        initial_rx_rates.append(initial_rate_rx_i)
    return initial_rx_rates

In [ ]:
#Esta función objetivo sera utilizada para resolver el sistema
def objective_function(param,experimental_data,experimental_molar_concentrations):
    #esta función devolverá la suma de las diferencias cuadradas entre las velocidades iniciales de reacción experimentales y las calculadas
    k1=param[0]
    calculated_initial_rx_rates=initial_reaction_rate(experimental_molar_concentrations,k1)
    #ahora extraeremos las velocidades iniciales de reacción de la lista calculada
    #ahora que tenemos los datos calculados, podemos calcular la suma de las diferencias cuadradas
    residuals = [calc - exp for calc, exp in zip(calculated_initial_rx_rates, experimental_data)]    #with this we put the elements in calculated_data and experimental_data into pairs and calculate the difference between them
    return np.array(residuals)  # least_squares minimizará la suma de los cuadrados de estos
#ahora que definimos nuestra función objetivo, podemos usar least_squares para optimizar los parámetros del modelo

In [ ]:
#Esta función esta encargada de optimizar el parametro k1
def optimize_model_parameters(experimental_rates,experimental_mole_concentrations,initial_param):
    
    bounds=([0], [np.inf])   #limites para el parametro
    result=least_squares(objective_function,initial_param, args=(experimental_rates, experimental_mole_concentrations), bounds=bounds)
    return result.x  #parametro optimizado

In [ ]:
##condiciones experimentales
experiment_masses=[[[0.0447,'mass'],[1.799,'mass'],[5.397,'mass'],[0,'mass'],[34.323,'mass']],[[0.0944,'mass'],[1.799,'mass'],[5.397,'mass'],[0,'mass'],[34.292,'mass']],[[1.2010,'mass'],[1.799,'mass'],[5.397,'mass'],[0,'mass'],[33.595,'mass']],[[0.0447,'mass'],[3.6,'mass'],[10.8,'mass'],[0,'mass'],[33.250,'mass']],[[0.0944,'mass'],[3.6,'mass'],[10.8,'mass'],[0,'mass'],[33.218,'mass']],[[0.6005,'mass'],[3.6,'mass'],[10.8,'mass'],[0,'mass'],[32.900,'mass']],[[1.2010,'mass'],[3.6,'mass'],[10.8,'mass'],[0,'mass'],[32.522,'mass']]]
experimental_initial_rx=[0.0201,0.0923,0.16237,0.05546,0.10678,0.175,0.1728]
components_for_optimization=['acetic acid','hydrogen peroxide','water','peracetic acid','hexane']
T=273.15
P=1
k1_to_optimize=[0.1]
experiment_concentrations=[]
for j in range(len(experiment_masses)):
    components_initial_values_to_evaluate=experiment_masses[j]
    experiment_moles=get_moles(components_for_optimization,components_initial_values_to_evaluate,T,P)
    experiment_concentrations.append(experiment_moles)
print(experiment_concentrations)
test=initial_reaction_rate(experiment_concentrations,k1_to_optimize[0])
print(test)
optimized_param=optimize_model_parameters(experimental_initial_rx,experiment_concentrations,k1_to_optimize)
print("Optimized Param: ",optimized_param)

[[0.0007443553882337896, 0.05288892913295083, 0.29957902402849135, 0.0, 0.39829250495733354], [0.0015719720055765041, 0.05288892913295083, 0.29957902402849135, 0.0, 0.3979327733588813], [0.01999934723196379, 0.05288892913295083, 0.29957902402849135, 0.0, 0.3898446145162608], [0.0007443553882337896, 0.10583665640835076, 0.5994910986673535, 0.0, 0.38584114995284036], [0.0015719720055765041, 0.10583665640835076, 0.5994910986673535, 0.0, 0.3854698141092768], [0.009999673615981895, 0.10583665640835076, 0.5994910986673535, 0.0, 0.38177966416386305], [0.01999934723196379, 0.10583665640835076, 0.5994910986673535, 0.0, 0.3773932595117676]]
[34.561287914825876, 17.15196153942831, 1.8570355105379432, 47.370966214979106, 23.217561327742388, 4.253377126907723, 2.3337987245403156]
Optimized Param:  [0.00020549]


## Solución del problema
### Ahora que tenemos el parámetro k1, contamos con todo lo necesario para resolver el sistema PFR
### Tenemos la siguiente definición de $-r_a$
$$
-r_a=r_1+r_2+r_3=\frac{k_1(C_A^{0.5}+C_B^{0.5})}{C_A}+\frac{k_2C_B^{1.5}}{C_A}+k_3(C_A^{0.3}+C_B^{0.3})
$$
### Donde debemos recordar que $C_B$ puede representarse mediante la siguiente ecuación
$$
C_B=\sqrt{C_A^2+2}
$$
### Ahora procedemos a obtener el volumen del sistema usando la siguiente ecuación
$$
\frac{C_{Ao}V}{F_{Ao}}=-\int_{C_{Ao}}^{C_{Af}} \frac{1}{-r_A} \, dc_A
$$
### Y el rendimiento global del sistema con una fórmula casi idéntica:
$$
\frac{1}{-(C_{Af}-C_{Ao})} \int_{C_{Ao}}^{C_{Af}} \frac{r_P}{-r_A} \, dc_A
$$

In [ ]:
##Asignación de los parametros
CAo=10
CAf=0.1
FAo=5
k1=0.00020549
k2=0.000016
k3=0.00029
##Volumen
def CB(CA):
    return np.sqrt(CA**2+2)
def rA(CA):
    CB_to_use=CB(CA)
    r1 = k1 * (CA**0.5 + CB_to_use**0.5) / CA
    r2 = k2 * (CB_to_use**1.5) / CA
    r3 = k3 * (CA**0.3 + CB_to_use**0.3)
    return (r1+r2+r3)
def value_to_integrate(CA):
    return 1/rA(CA)
answer=quad(value_to_integrate,CAo,CAf)
print(answer[0])
V=answer[0]*(-1)*FAo/CAo
print('Pfr volume: ', V, 'L')



-8205.169553160353
Pfr volume:  4102.584776580176 L


In [ ]:
##Rendimiento global
def instant_yield(CA):
    CB_to_use=CB(CA)
    r1 = k1 * (CA**0.5 + CB_to_use**0.5) / CA
    r2 = k2 * (CB_to_use**1.5) / CA
    r3 = k3 * (CA**0.3 + CB_to_use**0.3)
    return r1/(r1+r2+r3)
answ=quad(instant_yield,CAf,CAo) 
global_yield=answ[0]/(CAo-CAf)
print('Global yield: ',global_yield)

Global yield:  0.21043331409763252
